In [1]:
import pandas as pd
import numpy as np
import general_settings
from time import sleep

# isolation and quarantine

In [3]:
general_settings.file_url

'https://www.ag.ch/media/kanton_aargau/themen_1/coronavirus_1/daten_excel/Covid-19-Daten_Kanton_Aargau.xlsx'

In [4]:
#open sheet nr. 4
#sleep(20)
df_iso = pd.read_excel(general_settings.file_url, sheet_name=4)

In [5]:
df_iso


,Unnamed: 0,Contact Tracing,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
0,NaN,Infizierte Personen in Isolation,NaN,NaN,Kontaktpersonen in Quarantäne,NaN,NaN
1,NaN,Neue Fälle,Gesamtzahl aktuell betreuter Personen,Gesamtzahl seit Start Contact Tracing (11.05.2...,Neue Fälle,Gesamtzahl aktuell betreuter Personen,Gesamtzahl seit Start Contact Tracing (11.05.2...
2,2020-05-01 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-05-02 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-05-03 08:00:00,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
158,2020-10-18 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
159,2020-10-19 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
160,2020-10-20 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
161,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#rename and choose header
df_iso.iloc[1,0] = "date"
df_iso.columns = df_iso.iloc[1]
df_iso = df_iso.drop([0,1])

In [7]:
#formatting
df_iso["date"] = pd.to_datetime(df_iso["date"], errors="coerce")
df_iso.columns = ["date", "new_isolated", "total_curr_isolated", "total_isolated",
                 "new_quar", "total_curr_quar", "total_quar"]

#get rid of last row
df_iso.drop(df_iso.tail(1).index,inplace=True)

#if Monday (weekday == 0), take Friday as latest values
if general_settings.todays_weekday == 0:
    df_iso = df_iso[df_iso["date"] < general_settings.two_days_ago]
else:
    df_iso = df_iso[df_iso["date"] < general_settings.today]

In [8]:
#fill NaN values with previous value
df_iso = df_iso.fillna(method='ffill')

#get relevant rows and columns
df_iso_time = df_iso[["date", "total_curr_isolated", "total_curr_quar"]]

In [9]:
#formatting
df_iso_time["date"] = df_iso_time["date"].dt.normalize()
df_iso_time.columns = ["Datum", "Isolation", "Quarantäne"]

<ipython-input-9-6411bc132d73>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_iso_time["date"] = df_iso_time["date"].dt.normalize()


In [ ]:
df_iso_time

In [ ]:
#make a backup export of the current data
df_iso_time.to_csv("backups/iso_over_time/backup_{}.csv".format(general_settings.today))

#export to csv
df_iso_time.to_csv("iso_over_time.csv", index=False)